#Optimal placement to minimize the time it takes for a pallet rack machine to pick up items for different orders.

A machine that goes through a pallet rack to pick up items for an order can be seen as a traveling salesman problem.
Which path should the machine take to minimize the distance?
But we need also to take into an account that there could be several types of orders containing a different set of items, and that we can choose the placement of the items in the pallet rack.
We get a bilevel optimization problem. First must choose an item placement that in turn optimizes the length the machine has to travel to be able to safices all the orders.

In [5]:
!pip install python-tsp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


To solve the TSP I choose a dynamic programming approach since it both an exact algorithm and the number of pallet slots shouldn't be that big that time becomes a problem.

The TSP will have the euclidean distance as its edges. This could maybe be swapped for a optimal control "bang-bang" approach that takes in to an account the maximum acceleration/deceleration that the machine needs to do between each slot.

In [6]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import euclidean_distance_matrix

The order is a list of items that are tagged by a specific number. We also take into an account the expected number of each order each day. So if an order does not overlap much with the other orders but might be a very frequent order the algorithm should place that unique item closer to the machine's starting point.

In [7]:
order_1 = [9,4,3,5,7]
order_2 = [7,4,9]
order_3 = [7]

expt_order_1_amount = 35
expt_order_2_amount = 90
expt_order_3_amount = 85

orders = [[order_1,expt_order_1_amount],[order_2,expt_order_2_amount],[order_3,expt_order_3_amount]]

height = 3
width = 3

To update the placement a simple tabu search is introduced (a more advanced tabu search could be used for larger problems). It takes a vector of the placements (a flatten matrix) and swaps two adjacent items.

In [8]:
def arr_to_matrix(arr, height = height, width=width):
  return np.flip(np.asarray(arr).reshape((height,  width)), axis = 0)

def coord(arr,order):
  temp = []
  matrix = arr_to_matrix(arr)
  for i in order:
    res = np.where(matrix==i)
    temp.append([np.abs(int(res[0])-(height-1)),int(res[1])+1])
  return temp

def dist_matrix(nodes):
  nodes.reverse()
  nodes.append([0,0])
  nodes.reverse()
  sources = np.array(nodes)
  destinations = np.array(nodes)
  return euclidean_distance_matrix(sources, destinations)

def dp(distance_matrix):
  permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
  return distance

def neighbor_function(arry): #make better if many empty slots
  arr = arry.copy()
  point = np.random.randint(0, len(arr))
  if point == 0:
    dir = 1
  elif point == len(arr)-1:
    dir = 0
  else:
    dir = np.random.randint(0, 2)
  
  if dir == 0:
    arr[point],arr[point-1] = arr[point-1],arr[point]
  else:
    arr[point],arr[point+1] = arr[point+1],arr[point]
  return arr

def fitness(arr):
  fitness = 0
  for i in orders:
    nodes = coord(arr, i[0])
    distance_matrix = dist_matrix(nodes)
    fitness += i[1]*dp(distance_matrix)
  return fitness


def tabu_search(init_state, max_iter, tabu_size, neighbor_amount):
  best = init_state
  best_fit = fitness(init_state)

  current = init_state
  current_fit = fitness(init_state)
  
  tabulist = [init_state]

  for j in range(max_iter):
    tresh = np.inf
    neighbors = []
    
    for _ in range(neighbor_amount):
      neighbors.append(neighbor_function(current))

    for i in neighbors:
      if i not in tabulist:
        
        temp_fit = fitness(i)
        if temp_fit < tresh:
          
          current = i
          current_fit = temp_fit
          thresh = temp_fit
          

    if current_fit < best_fit:
      best = current
      best_fit = current_fit

    tabulist.append(current)

    if len(tabulist)>tabu_size:
      tabulist.pop(0)
  return best, best_fit

In [9]:
a,b = tabu_search([1,2,3,4,5,6,7,8,9], 1000, 3, 5)

In [10]:
init = [1,2,3,4,5,6,7,8,9]
print('Init layout')
print(arr_to_matrix(init))
print('Init fitness', fitness(init))

Init layout
[[7 8 9]
 [4 5 6]
 [1 2 3]]
Init fitness 1442.1603413171142


In [11]:
print('Optimized layout')
print(arr_to_matrix(a))
print('Optimized fitness', b)

Optimized layout
[[2 6 3]
 [9 5 1]
 [7 4 8]]
Optimized fitness 886.8157698057664
